# **Individual Planning DSCI 100 Project**

# 1: Data Description

### Data Collection Method:
- The data of both datasets were collected by the UBC research group, Pacific Laboratory for Artificial Intelligence through recording player behaviour in MineCraft.

### players.csv:
- The players dataframe has 196 observations, with 7 variables

| Variable | Description | Data type |
|----------|-------------|-----------|
| experience | Self-identifying Level of Experience | chr|
| subscribe  | TRUE if player is Subscribed to Newsletter| lgl |
| hashedEmail | Hashed Version of Player Email | chr |
| played_hours | Hours spent in game | dbl |
| name | Name of player | chr |
| gender | Gender of player | chr |
| age | Age of player | dbl |

### summary statistics
- From the `summary()` function:

| Variable | Min | Max | Mean |
|----------|-----|-----|------|
| played_hours | 0.00 | 223.10 | 5.85|
|age | 9.00 | 58.00 | 21.14 |


In [57]:
player_pro <- player_data |>
    filter(experience == "Pro")|>
    nrow()

player_veteran <- player_data |>
    filter(experience == "Veteran") |>
    nrow()

player_amateur <- player_data |>
    filter(experience == "Amateur")|>
    nrow()

player_regular <- player_data |>
    filter(experience == "Regular")|>
    nrow()

player_beginner <- player_data |>
    filter(experience == "Beginner")|>
    nrow()

player_rows <- nrow(player_data)

beginner_pc <- round((player_beginner/player_rows), 2)
amateur_pc <- round((player_amateur/player_rows), 2)
regular_pc <- round((player_regular/player_rows), 2)
veteran_pc <- round((player_veteran/player_rows), 2)
pro_pc <- round((player_pro/player_rows), 2)

The proportion of self-identifying levels of experience is as follows:

- Beginner players: 0.18
- Amateur players: 0.32
- Regular players: 0.18
- Veteran players: 0.24
- Pro players: 0.07

In [59]:
true_players <- player_data |>
    filter(subscribe == TRUE) |>
    nrow()

false_players <- player_data |>
    filter(subscribe == FALSE) |>
    nrow()

true_pc <- round((true_players/player_rows), 2)
false_pc <- round((false_players/player_rows), 2)

The proportion of players subscribed to the game newsletter is as follows:
- True: 0.73
- False: 0.27

### sessions.csv
- The sessions dataframe has 1535 observations, with 5 observations
| Variable | Description | Data type |
|----------|-------------|-----------
| hashedEmail | Hashed Version of Player Email | chr |
| start_time | Start Time of Session | chr|
| end_time | End Time of Session | chr |
| original_start_time | Start Time Represented in UNIX time | dbl |
| original_end_time | End Time Represented in UNIX time | dbl |

### summary statistics
- From the `summary()` function:

| Variable | Min | Max | Mean |
|----------|-----|-----|------|
| original_start | 1.71e+12 | 1.73e+12 | 1.72e+12 |
| orginal_end_time | 1.71e+12 |1.73e+12 | 1.72e+12 |

## Issues
- Data is given in 2 separate .csv files, making the data difficult to interpret between the files. This can be solved through merging the datasets into one through their mutual variable (hashedEmail), using `inner_join`
- In the `players.csv` dataframe, experience can be changed to fct data types. Age can also be changed to an int data type.

# 2: Questions
**Broad question:** What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

**Specific question:** Can the status of subscribing to a game-related newsletter be predicted from Age, gender, and played_hours?

# 3: Exploratory Data Analysis + Visualization
Using the tidyverse package, the datasets can be loaded into R with `read_csv`.

In [66]:
library(tidyverse)

player_data <- read_csv("data/players.csv") 
sessions_data <- read_csv("data/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [67]:
player_summary <- summary(player_data)
player_summary

sessions_summary <- summary(sessions_data)
sessions_summary

  experience        subscribe       hashedEmail         played_hours    
 Length:196         Mode :logical   Length:196         Min.   :  0.000  
 Class :character   FALSE:52        Class :character   1st Qu.:  0.000  
 Mode  :character   TRUE :144       Mode  :character   Median :  0.100  
                                                       Mean   :  5.846  
                                                       3rd Qu.:  0.600  
                                                       Max.   :223.100  
                                                                        
     name              gender               Age       
 Length:196         Length:196         Min.   : 9.00  
 Class :character   Class :character   1st Qu.:17.00  
 Mode  :character   Mode  :character   Median :19.00  
                                       Mean   :21.14  
                                       3rd Qu.:22.75  
                                       Max.   :58.00  
                               

 hashedEmail         start_time          end_time         original_start_time
 Length:1535        Length:1535        Length:1535        Min.   :1.712e+12  
 Class :character   Class :character   Class :character   1st Qu.:1.716e+12  
 Mode  :character   Mode  :character   Mode  :character   Median :1.719e+12  
                                                          Mean   :1.719e+12  
                                                          3rd Qu.:1.722e+12  
                                                          Max.   :1.727e+12  
                                                                             
 original_end_time  
 Min.   :1.712e+12  
 1st Qu.:1.716e+12  
 Median :1.719e+12  
 Mean   :1.719e+12  
 3rd Qu.:1.722e+12  
 Max.   :1.727e+12  
 NA's   :2          

In [29]:
joined <- inner_join(player_data, sessions_data)
joined

Joining with `by = join_by(hashedEmail)`


experience,subscribe,hashedEmail,played_hours,name,gender,Age,start_time,end_time,original_start_time,original_end_time
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,08/08/2024 00:21,08/08/2024 01:35,1.72308e+12,1.72308e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,09/09/2024 22:30,09/09/2024 22:37,1.72592e+12,1.72592e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,08/08/2024 02:41,08/08/2024 03:25,1.72308e+12,1.72309e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,10/09/2024 15:07,10/09/2024 15:29,1.72598e+12,1.72598e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,05/05/2024 22:21,05/05/2024 23:17,1.71495e+12,1.71495e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,06/04/2024 22:24,06/04/2024 23:33,1.71244e+12,1.71245e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,20/04/2024 20:46,20/04/2024 21:48,1.71365e+12,1.71365e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,15/06/2024 16:37,15/06/2024 18:37,1.71847e+12,1.71848e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,05/05/2024 23:40,06/05/2024 00:55,1.71495e+12,1.71496e+12


In [ ]:
pclass 